![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)


This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Practical Overview of Selected Text Analytics Methods*](https://ladal.edu.au/textanalysis.html). 



***


**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial.


In [ ]:
# activate packages
library(quanteda)
library(dplyr)
library(stringr)
library(tidytext)
library(ggplot2)
library(quanteda.textplots)
library(quanteda.textstats)
library(udpipe)
library(ggplot2)
library(textplot)


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("quanteda")`. Then, you simply run this command and R will install the package you specified.


## Loading and processing textual data

For this tutorial, the default data represents the text of Lewis Caroll's  *Alice's Adventures in Wonderland* which we download from the [GitHub data repository of the *Language Technology and Data Analysis Laboratory* (LADAL)](https://slcladal.github.io/data). 

***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytext1 <- scan(file = "linguistics01.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
mytext2 <- scan(file = "linguistics02.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
# inspect
mytext1; mytext2


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***


We begin by loading the data which represents the text of Lewis Caroll's  *Alice's Adventures in Wonderland*. 


In [ ]:
# load text
text <- base::readRDS(url("https://slcladal.github.io/data/alice.rda", "rb"))
# inspect data
head(text)


The data still consists of short text snippets which is why we collapse these snippets and then split the collapsed data into chapters. 



In [ ]:
# combine and split into chapters
text_chapters <- text %>%
  # paste all texts together into one long text
  paste0(collapse = " ") %>%
  # replace Chapter I to Chapter XVI with qwertz 
  stringr::str_replace_all("(CHAPTER [XVI]{1,7}\\.{0,1}) ", "qwertz\\1") %>%
  # convert text to lower case
  tolower() %>%  
  # split the long text into chapters
  stringr::str_split("qwertz") %>%
  # unlist the result (convert into simple vector)
  unlist()
# inspect data
text_chapters %>%
  substr(start=1, stop=500) %>%
  as.data.frame() %>%
  head()


Once we have split the data into chapters, we perform the concordancing and extract the KWICs. To create these kwics, we use the `kwic` function from the `quanteda` package. This function takes the data (x), the search pattern (pattern), and the window size as its main arguments. 

To start with, we generate kwics for the term *alice* as shown below. 


In [ ]:
# create kwic
kwic_alice <- quanteda::kwic(x = text_chapters, # define text(s) 
                         # define pattern
                          pattern = "alice",
                         # define window size
                          window = 5) %>%
  # convert into a data frame
  as.data.frame() %>%
  # remove superfluous columns
  dplyr::select(-to, -from, -pattern)
# inspect data
kwic_alice %>%
  head()


We can also use regular expressions in our search to extract not only *alice* but also more complex and even vague patterns. Vague means that only part of the pattern is specified. For instance, maybe only *walk* is specified and now we want all words containing this sequence including *walking*, *walker*, *walked*, and *walks*. To retrieve such vague patterns, we need to use so-called *regular expressions*. Also, when using a regular expression in the `pattern` argument, we need to specify the `valuetype` as `regex` (as shown below).



In [ ]:
# create kwic
kwic_walk <- quanteda::kwic(x = text_chapters, 
                          pattern = "walk.*",
                          window = 5,
                          valuetype = "regex") %>%
  # convert into a data frame
  as.data.frame() %>%
  # remove superfluous columns
  dplyr::select(-to, -from, -pattern)
# inspect data
kwic_walk %>%
  head()


When search for expressions that represent phrase and that consists out of several elements such as *poor alice*, we also need to specify that we are looking for a phrase in the pattern argument. 



In [ ]:
# create kwic
kwic_pooralice <- quanteda::kwic(x = text_chapters, 
                          pattern = quanteda::phrase("poor alice"),
                          window = 5) %>%
  # convert into a data frame
  as.data.frame() %>%
  # remove superfluous columns
  dplyr::select(-to, -from, -pattern)
# inspect data
kwic_pooralice %>%
  head()


We could now continue and analyze how the phrase *poor alice* is used or perform similar analyses.

# Word Frequency

Almost all methods used in text analytics rely on frequency information. Thus, fending out out frequent words are in a text is a fundamental technique in text analytics. In fact, frequency information lies at the very core of Text Analysis. Such frequency information often comes in the form of word frequency lists, i.e. lists of word forms and their frequency in a given text or collection of texts.  

As extracting word frequency lists is very important, we will now We will now extract a frequency list from a corpus.

In a first step, we load a corpus, convert everything to lower case, remove non-word symbols (including punctuation), and split the corpus data into individual words.


In [ ]:
# load and process corpus
text_words <- text  %>%
  # convert everything to lower case
  tolower() %>%
  # remove non-word characters
  str_replace_all("[^[:alpha:][:space:]]*", "")  %>%
  tm::removePunctuation() %>%
  stringr::str_squish() %>%
  stringr::str_split(" ") %>%
  unlist()
# inspect data
text_words %>%
  head()


Now that we have a vector of words, we can easily create a table representing a word frequency list (as shown below).



In [ ]:
# create table
wfreq <- text_words %>%
  table() %>%
  as.data.frame() %>%
  arrange(desc(Freq)) %>%
  dplyr::rename(word = 1,
                frequency = 2)
# inspect data
wfreq %>%
  head()


The most frequent words are all function words which are often not meaningful or useful for an analysis. Thus, we now remove these function words (also called *stopwords*) from the frequency list and inspect the list without stopwords.



In [ ]:
# create table wo stopwords
wfreq_wostop <- wfreq %>%
  anti_join(stop_words, by = "word") %>%
  dplyr::filter(word != "")
# inspect data
wfreq_wostop %>%
  head() 


Such word frequency lists can be visualized in various ways. The most common way to visualize word frequency lists is in the form of bargraphs.



In [ ]:
wfreq_wostop %>%
  head(10) %>%
  ggplot(aes(x = reorder(word, -frequency, mean), y = frequency)) +
  geom_bar(stat = "identity") +
  labs(title = "10 most frequent non-stop words in \n the example text ",
       x = "") +
  theme(axis.text.x = element_text(angle = 45, size = 12, hjust = 1))


## Wordclouds

Alternatively, word frequency lists can be visualized, although less informative, as word clouds. 


In [ ]:
# create wordcloud
text %>%
  quanteda::corpus() %>%
  quanteda::tokens(remove_punct = TRUE) %>%
  quanteda::tokens_remove(stopwords("english")) %>%
  quanteda::dfm() %>%
  quanteda.textplots::textplot_wordcloud()


Another variant of word clouds, so-called *comparison clouds*, Word lists can be used to determine differences between texts. For instance, we can load different texts and check whether they differ with respect to word frequencies. To show this, we load Herman Melville's *Moby Dick*, George Orwell's *1984*, and we also use  in the example text . 

In a first step, we load these texts and collapse them into single documents.


In [ ]:
# load data
darwin_sep <- base::readRDS(url("https://slcladal.github.io/data/darwin.rda", "rb"))
darwin <- darwin_sep %>%
  paste0(collapse = " ")
orwell_sep <- base::readRDS(url("https://slcladal.github.io/data/orwell.rda", "rb"))
orwell <- orwell_sep %>%
  paste0(collapse = " ")
melville_sep <- base::readRDS(url("https://slcladal.github.io/data/melville.rda", "rb"))
melville <- melville_sep %>%
  paste0(collapse = " ")


Now, we generate a corpus object from these texts and create a variable with the author name.



In [ ]:
corp_dom <- quanteda::corpus(c(darwin, orwell, melville)) 
attr(corp_dom, "docvars")$Author = c("Darwin", "Orwell", "Melville")


Now, we can remove so-called *stopwords* (non-lexical function words) and punctuation and generate the comparison cloud.



In [ ]:
corp_dom  %>%
    quanteda::tokens(remove_punct = TRUE) %>%
    quanteda::tokens_remove(stopwords("english")) %>%
    quanteda::dfm() %>%
    quanteda::dfm_group(groups = corp_dom$Author) %>%
    quanteda::dfm_trim(min_termfreq = 200, verbose = FALSE) %>%
    quanteda.textplots::textplot_wordcloud(comparison = TRUE)


## Frequency changes

We can also investigate the use of the term *organism* across chapters in Darwin's *Origin*. In a first step, we extract the number of words in each chapter.


In [ ]:
# extract number of words per chapter
Words <- text_chapters %>%
  stringr::str_split(" ")  %>%
  lengths()
# inspect data
Words


Next, we extract the number of matches in each chapter.



In [ ]:
# extract number of matches per chapter
Matches <- text_chapters %>%
  stringr::str_count("alice")
# inspect the number of matches per chapter
Matches


Now, we extract the names of the chapters and create a table with the chapter names and the relative frequency of matches per 1,000 words.



In [ ]:
# extract chapters
Chapters <- paste0("chapter ", 0:(length(text_chapters)-1))
Chapters


In [ ]:
# create table of results
tb <- data.frame(Chapters, Matches, Words) %>%
  dplyr::mutate(Frequency = round(Matches/Words*1000, 2))
# inspect data
tb %>%
  as.data.frame() %>%
  head() 


We can now visualize the relative frequencies of our search word per chapter.



In [ ]:
# create plot
ggplot(tb, aes(x = Chapters, y = Frequency, group = 1)) + 
  geom_smooth(color = "purple") +
  geom_line(color = "darkgray") +         
  guides(color=guide_legend(override.aes=list(fill=NA))) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))+
  scale_y_continuous(name ="Relative Frequency (per 1,000 words)")


## Dispersion plots

To show when in a text or in a collection of texts certain terms occur, we can use *dispersion plots*. The `quanteda` package offers a very easy-to-use function `textplot_xray` to generate dispersion plots.


In [ ]:
# add chapter names
names(text_chapters) <- Chapters
# generate corpus from chapters
text_corpus <- quanteda::corpus(text_chapters)
# generate dispersion plots
quanteda.textplots::textplot_xray(kwic(text_corpus, pattern = "alice"),
                                  kwic(text_corpus, pattern = "hatter"),
                                  sort = T)


We can modify the plot by saving it into an object and then use `ggplot` to modify it appearance.



In [ ]:
# generate and save dispersion plots
dp <- quanteda.textplots::textplot_xray(kwic(text_corpus, pattern = "alice"),
                                        kwic(text_corpus, pattern = "cat"))
# modify plot
dp + aes(color = keyword) + 
  scale_color_manual(values = c('red', 'blue')) +
  theme(legend.position = "none")


## Over- and underuse

Frequency information can also tell us something about the nature of a text. For instance, private dialogues will typically contain higher rates of second person pronouns compared with more format text types, such as, for instance, scripted monologues like speeches. For this reason, word frequency lists can be used in text classification and to determine the formality of texts.

As an example, below you find the number of the second person pronouns *you* and *your* and the number of all words except for these second person pronouns in private dialogues compared with scripted monologues in the Irish component of the *International Corpus of English* (ICE). In addition, the tables shows  the percentage of second person pronouns in both text types to enable seeing  whether private dialogues contain more of these second person pronouns than scripted monologues (i.e. speeches).


In [ ]:
numbers <- matrix(c("you, your", "6761", "659", "Other words",	"259625",	
                    "105295", "Percent",	"2.60",	"0.63"), byrow = T, nrow = 3)
colnames(numbers) <- c("", "Private dialogues", "Scripted monologues")
# inspect data
ndf <- numbers %>%
  as.data.frame()
colnames(ndf)[1] <- "."
ndf


This simple example shows that second person pronouns make up 2.6 percent of all words that are used in private dialogues while they only amount to 0.63 percent in scripted speeches. A handy way to present such differences visually are association and mosaic plots.



In [ ]:
d <- matrix(c(6761, 659, 259625, 105295), nrow = 2, byrow = T)
colnames(d) <- c("D", "M")
rownames(d) <- c("you, your", "Other words")
assocplot(d)


Bars above the dashed line indicate relative overuse while bars below the line suggest relative under-use. Therefore, the association plot indicates under-use of *you* and *your* and overuse of *other words* in monologues while the opposite trends holds true for dialogues, i.e. overuse of *you* and *your* and under-use of *Other words*. 

# N-grams, Collocations, and  Keyness

Collocation refers to the co-occurrence of words. A typical example of a collocation is *Merry Christmas* because the words merry and Christmas occur together more frequently together than would be expected by chance, if words were just randomly stringed together.  

N-grams are related to collocates in that they represent words that occur together (bi-grams are two words that occur together, tri-grams three words and so on). Fortunately, creating N-gram lists is very easy. We will use the example text to create a bi-gram list. We can simply take each word and combine it with the following word.


In [ ]:
# create data frame
text_bigrams <- data.frame(text_words[1:length(text_words)-1], 
                       text_words[2:length(text_words)]) %>%
  dplyr::rename(Word1 = 1,
                Word2 = 2) %>%
  dplyr::mutate(Bigram = paste0(Word1, " ", Word2)) %>%
  dplyr::group_by(Bigram) %>%
  dplyr::summarise(Frequency = n()) %>%
  dplyr::arrange(-Frequency)
# inspect data
text_bigrams %>%
  head()


Both N-grams and collocations are not only an important concept in language teaching but they are also fundamental in Text Analysis and many other research areas working with language data. Unfortunately, words that collocate do not have to be immediately adjacent but can also encompass several slots which makes it harder to retrieve of collocates that are not adjacent- We will find out how to identify non-adjacent collocates in the next section.

## Finding collocations

There are various techniques for identifying collocations. To identify collocations without having a pre-defined target term, we can use the `textstat_collocations` function from the `quanteda.textstats` package.

However, before we can apply that function and start identifying collocations, we need to process the data to which we want to apply this function. In the present case, we will apply that function to the sentences  in the example text  which we extract in the code chunk below.


In [ ]:
text_sentences <- text %>%
  tolower() %>%
  paste0(collapse= " ") %>%
  stringr::str_split(fixed(".")) %>%
  unlist() %>%
  tm::removePunctuation() %>%
  stringr::str_squish()
# inspect data
text_sentences %>%
  head() 


From the output shown above, we also see that splitting texts simply by full stops is not optimal as it produces some unwarranted artifacts like the "sentences" that consist of single characters. Fortunately, these errors do not really matter in the case of our example.

Now that we have split the example text  into sentences, we can tokenize these sentences and apply the `textstat_collocations` function which identifies collocations.


In [ ]:
# create a token object
text_tokens <- tokens(text_sentences, remove_punct = TRUE) %>%
  tokens_remove(stopwords("english"))
# extract collocations
text_coll <- textstat_collocations(text_tokens, size = 2, min_count = 20)
# inspect data
text_coll %>%
  head() 


The resulting table shows collocations  in the example text  descending by collocation strength.

## Visualizing Collocation Networks

Network graphs are a very useful and flexible tool for visualizing relationships between elements such as words, personas, or authors. This section shows how to generate a network graph for collocations of the term *alice* using the `quanteda` package.

In a first step, we generate a document-feature matrix based on the sentences  in the example text. A document-feature matrix shows how often elements (here these elements are the words that occur in the *Origin*) occur in a selection of documents (here these documents are the sentences in the example text).


In [ ]:
# create document-feature matrix
text_dfm <- text_sentences %>% 
    quanteda::dfm(remove = stopwords('english'), remove_punct = TRUE) %>%
    quanteda::dfm_trim(min_termfreq = 10, verbose = FALSE)
# inspect data
text_dfm[1:6, 1:6]


As we want to generate a network graph of words that collocate with the term *organism*, we use the `calculateCoocStatistics` function to determine which words most strongly collocate with our target term (*organism*).  



In [ ]:
# load function for co-occurrence calculation
source("https://slcladal.github.io/rscripts/calculateCoocStatistics.R")
# define term
coocTerm <- "alice"
# calculate co-occurrence statistics
coocs <- calculateCoocStatistics(coocTerm, text_dfm, measure="LOGLIK")
# inspect results
coocs[1:20]


We now reduce the document-feature matrix to contain only the top 20 collocates of *organism* (plus our target word *organism*).



In [ ]:
redux_dfm <- dfm_select(text_dfm, 
                        pattern = c(names(coocs)[1:20], "alice"))
# inspect data
redux_dfm[1:6, 1:6]


Now, we can transform the document-feature matrix into a feature-co-occurrence matrix as shown below. A feature-co-occurrence matrix shows how often each element in that matrix co-occurs with every other element in that matrix.



In [ ]:
tag_fcm <- fcm(redux_dfm)
# inspect data
tag_fcm[1:6, 1:6] 


Using the feature-co-occurrence matrix, we can generate the network graph which shows the terms that collocate with the target term *alice* with the edges representing the co-occurrence frequency. To generate this network graph, we use the `textplot_network` function from the `quanteda.textplots` package.



In [ ]:
# generate network graph
textplot_network(tag_fcm, 
                 min_freq = 1, 
                 edge_alpha = 0.1, 
                 edge_size = 5,
                 edge_color = "purple",
                 vertex_labelsize = log(rowSums(tag_fcm))*2)


## Keyness

Another common method that can be used for automated text summarization is keyword extraction. Keyword extraction builds on identifying words that are particularly associated  with a certain text. In other words, keyness analysis aims to identify words that are particularly indicative of the content of a certain text.

Below, we identify key words for Charles Darwin's *Origin*, Herman Melville's *Moby Dick*, and George Orwell's *1984*. We start by creating a weighted document feature matrix from the corpus containing the three texts. 

In order to create a corpus, we use the text objects that consist out of many different elements rather than the objects which contained the collapsed texts that we used above. Thus, in a first step, we create a corpus of the texts.


In [ ]:
corp_dom <- quanteda::corpus(c(darwin_sep, orwell_sep, melville_sep)) 
attr(corp_dom, "docvars")$Author = c(rep("Darwin", length(darwin_sep)), 
                                     rep("Orwell", length(orwell_sep)),
                                     rep("Melville", length(melville_sep)))


Next, we generate the document feature matrix and we clean it by removing stopwords and selected other words. In addition, we group the documents feature matrix by author.



In [ ]:
dfm_authors <- corp_dom %>%
  quanteda::tokens(remove_punct = TRUE) %>%
  quanteda::tokens_remove(quanteda::stopwords("english")) %>%
  quanteda::tokens_remove(c("now", "one", "like", "may", "can")) %>%
  quanteda::dfm() %>%
  quanteda::dfm_group(groups = Author) %>%
  quanteda::dfm_weight(scheme = "prop")


In a next step, we use the `textstat_frequency` function from the `quanteda` package to extract the most frequent non-stopwords in the three texts.
 


In [ ]:
# Calculate relative frequency by president
freq_weight <- quanteda.textstats::textstat_frequency(dfm_authors, 
                                                      n = 10,
                                                      groups = dfm_authors$Author)
# inspect data
freq_weight %>%
  as.data.frame() %>%
  head()


Now, we can simply plot the most common words and most indicative non-stop words in the three texts.
 


In [ ]:
ggplot(freq_weight, aes(nrow(freq_weight):1, frequency)) +
     geom_point() +
     facet_wrap(~ group, scales = "free") +
     coord_flip() +
     scale_x_continuous(breaks = nrow(freq_weight):1,
                        labels = freq_weight$feature) +
     labs(x = NULL, y = "Relative frequency")


# Text Classification

Text classification refers to methods that allow to classify a given text to a predefined set of languages, genres, authors, or the like. Such classifications are typically based on the relative frequency of word classes, key words, phonemes, or other linguistic features such as average sentence length, words per line, etc.

As with most other methods that are used in text analysis, text classification typically builds upon a training set that is already annotated with the required tags. Training sets and the features that are derived from these training sets can be created by oneself or one can use build in training sets that are provided in the respective software packages or tools. 

In the following, we will use the frequency of phonemes to classify a text. In a first step, we read in a German text, and split it into phonemes.


In [ ]:
# read in German text
German <- readLines("https://slcladal.github.io/data/phonemictext1.txt") %>%
  stringr::str_remove_all(" ") %>%
  stringr::str_split("") %>%
  unlist()
# inspect data
head(German, 20)


We now do the same for three other texts - an English and a Spanish text as well as one text in a language that we will determine using classification.



In [ ]:
# read in texts
English <- readLines("https://slcladal.github.io/data/phonemictext2.txt")
Spanish <- readLines("https://slcladal.github.io/data/phonemictext3.txt")
Unknown <- readLines("https://slcladal.github.io/data/phonemictext4.txt")
# clean, split texts into phonemes, unlist and convert them into vectors
English <- as.vector(unlist(strsplit(gsub(" ", "", English), "")))
Spanish <- as.vector(unlist(strsplit(gsub(" ", "", Spanish), "")))
Unknown <- as.vector(unlist(strsplit(gsub(" ", "", Unknown), "")))
# inspect data
head(English, 20)


We will now create a table that represents the phonemes and their frequencies in each of the 4 texts. In addition, we will add the language and simply the column names.



In [ ]:
# create data tables
German <- data.frame(names(table(German)), as.vector(table(German)))
English <- data.frame(names(table(English)), as.vector(table(English)))
Spanish <- data.frame(names(table(Spanish)), as.vector(table(Spanish)))
Unknown <- data.frame(names(table(Unknown)), as.vector(table(Unknown)))
# add column with language
German$Language <- "German"
English$Language <- "English"
Spanish$Language <- "Spanish"
Unknown$Language <- "Unknown"
# simplify column names
colnames(German)[1:2] <- c("Phoneme", "Frequency")
colnames(English)[1:2] <- c("Phoneme", "Frequency")
colnames(Spanish)[1:2] <- c("Phoneme", "Frequency")
colnames(Unknown)[1:2] <- c("Phoneme", "Frequency")
# combine all tables into a single table
classdata <- rbind(German, English, Spanish, Unknown) 
# inspect data
classdata %>%
  as.data.frame() %>%
  head() 


Now, we group the data so that we see, how often each phoneme is used in each language.



In [ ]:
# convert into wide format
classdw <- classdata %>%
  tidyr::spread(Phoneme, Frequency) %>%
  replace(is.na(.), 0)
# inspect data
classdw[, 1:6] %>%
  as.data.frame() %>%
  head() 


Now, we need to transform the data again, so that we have the frequency of each phoneme by language as the classifier will use "Language" as the dependent variable and the phoneme frequencies as predictors.



In [ ]:
numvar <- colnames(classdw)[2:length(colnames(classdw))]
classdw[numvar] <- lapply(classdw[numvar], as.numeric)
# function for normalizing numeric variables
normalize <- function(x) { (x-min(x))/(max(x)-min(x))   }
# apply normalization
 classdw[numvar] <- as.data.frame(lapply(classdw[numvar], normalize))
# inspect data
classdw[, 1:6] %>%
  as.data.frame() %>%
  head() 


Before turning to the actual classification, we will use a cluster analysis to see which texts the unknown text is most similar with.



In [ ]:
# remove language column
textm <- classdw[,2:ncol(classdw)]
# add languages as row names
rownames(textm) <- classdw[,1]
# create distance matrix
distmtx <- dist(textm)
# perform clustering
clustertexts <- hclust(distmtx, method="ward.D")  
# visualize cluster result
plot(clustertexts, hang = .25,main = "")           


According to the cluster analysis, the unknown text clusters together with the English texts which suggests that the unknown text is likely to be English.

Before we begin with the actual classification, we will split the data so that we have one data set without "Unknown" (this is our training set) and one data set with only "Unknown" (this is our test set).


In [ ]:
# create training set
train <- classdw %>%
  filter(Language != "Unknown")
# create test set
test <- classdw %>%
  filter(Language == "Unknown")
# inspect data
classdw[, 1:6] %>%
  as.data.frame() %>%
  head()


Finally, we can apply our classifier to our data. The classifier we use is a k-nearest neighbor classifier as the underlying function will classify an unknown element given its proximity to the clusters in the training set.



In [ ]:
# set seed for reproducibility
set.seed(12345)
# apply k-nearest-neighbor (knn) classifier
prediction <- class::knn(train[,2:ncol(train)], 
                         test[,2:ncol(test)], 
                         cl = train[, 1], 
                         k = 3)
# inspect the result
prediction


Based on the frequencies of phonemes in the unknown text, the knn-classifier predicts that the unknown text is English. This is in fact true as the text is a subsection of the Wikipedia article for Aldous Huxley's *Brave New World*. The training texts were German, English, and Spanish translations of a subsection of Wikipedia's article for Hermann Hesse's *Steppenwolf*. 


# Part-of-Speech tagging

A very common procedure to add information to texts is to part-of-speech tag the data, which means to determine to what type of word a specific word belongs. Below, we will add pos-tags to a short English text.

We start by loading  a text


In [ ]:
# load text
text  <- base::readRDS(url("https://slcladal.github.io/data/orwell.rda", "rb")) %>%
  .[1:10] %>%
  paste0(collapse = " ")
# inspect
substr(text, 1, 200)


Now that we have a text that we can work with, we will download a pre-trained language model.



In [ ]:
# download language model
m_eng	<- udpipe::udpipe_download_model(language = "english-ewt")


If you have downloaded a model it will by default be stored in the main directory.



In [ ]:
# load language model from your computer after you have downloaded it once
m_eng <- udpipe_load_model("english-ewt-ud-2.5-191206.udpipe")


We can now use the model to annotate out text.



In [ ]:
# tokenise, tag, dependency parsing
text_anndf <- udpipe::udpipe_annotate(m_eng, x = text) %>%
  as.data.frame() %>%
  dplyr::select(-sentence)
# inspect
head(text_anndf, 10)


It can be useful to extract only the words and their pos-tags and convert them back into a text format (rather than a tabular format). 



In [ ]:
tagged_text <- paste(text_anndf$token, "/", text_anndf$xpos, collapse = " ", sep = "")
# inspect tagged text
substr(tagged_text, 1, 200)


We could use the pos-tagged data to study differences in the distribution of word classes across different registers. or to find certain syntactic patterns in a collection of texts. 

# Names Entity Recognition 

Named Entity Recognition (NER) (also referred to as *named entity extraction* or simply as *entity extraction*) is a text analytic method which allows us to automatically identify or extract named entities from text(s) such as persons, locations, brands, etc. 

As such, NER is a process during which textual elements which have characteristics that are common to proper nouns (locations, people, organizations, etc.) rather than other parts of speech, e.g. non-sentence initial capitalization, are extracted from texts. Retrieving entities is common in automated summarization and in Topic Modeling. NER can be achieved by simple feature extraction (e.g. extract all non-sentence initial capitalized words) or with the help of training sets. Using training sets, i.e. texts that are annotated for entities and non-entities, achieves better results when dealing with unknown data and data with inconsistent capitalization.

Here, we will make use of the results obtained form the pos-tagging and simply extract terms that have been tagged as 


In [ ]:
# tokenise, tag, dependency parsing
ner_df <- text_anndf %>%
  dplyr::filter(upos == "PROPN") %>%
  dplyr::select(token_id, token, lemma, upos, feats)
# inspect
head(ner_df)


The results would now have to be processed further and could be categorized into person, location, dates, entities, for example. However, this should already give you an idea and get you started.

# Dependency Parsing Using UDPipe

In addition to pos-tagging, we can also generate plots showing the syntactic dependencies of the different constituents of a sentence. For this, we generate an object that contains a sentence (in this case, the sentence *John gave Mary a kiss*), and we then plot (or visualize) the dependencies using the `textplot_dependencyparser` function.  


In [ ]:
# parse text
sent <- udpipe::udpipe_annotate(m_eng, x = "John gave Mary a kiss.") %>%
  as.data.frame()
# inspect
head(sent)


We now generate the plot.



In [ ]:
# generate dependency plot
dplot <- textplot::textplot_dependencyparser(sent, size = 3) 
# show plot
dplot


Dependency parsing cane be used, e.g. to study who is the agent versus who is the patient of certain actions such as crimes or other activities.



[Back to LADAL](https://ladal.edu.au)
